# Global indicator project
## Get and save study region walkable street network, load OSM basic network stats

This notebook describes the process to get street networks from OpenStreetMap using OSMnx, a Python library that lets you download spatial geometries and construct, project, and visualize street networks from OpenStreetMap's API. This process obtains OpenStreetMap network from study region buffer polygon. The network data are saved to local data folder.

In [1]:
# Libraries used for OSMnx analyses and output
import networkx as nx
import time 
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import geopandas as gpd

from shapely.geometry import shape,Point, LineString, Polygon


ox.config(use_cache=True, log_console=True)
ox.__version__

'0.9'

### Set up project config

In [2]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

suffix = '_201905' # output data time

buffer_dist = 1e4 #study region buffer 10km

data_folder = '../data/Maricopa_County'

OSM_folder = '../data/OSM'

shape_filename = 'MC_Boundary.shp'

filepath = data_folder + '/' + shape_filename

## Get pedestrain street network and save to local folder

In [5]:
# Extract complete OSM network and save local graph and edge shapefile: "all (non-private) OSM streets and paths"
def Save_OSM_G_gdf(polygon, network_type= 'walk', place=place, suffix=suffix, folder=OSM_folder):    
    """
    save a graphml and GeoDataFrame from a single polygon.
    Parameters
    ----------
    polygon : shapely Polygon or MultiPolygon
        the shape to get network data within. coordinates should be in units of
        latitude-longitude degrees.
    network_type : string
        what type of street network to get. Default type is pedestrain network
    place: string
        place name
    suffix: string
        output data date
    folder : string
        where to save the shapefile, specify local folder path for OSM resource
    Returns
    -------
    none

    """
    G = ox.graph_from_polygon(polygon, network_type=network_type, retain_all = True)
    #save network as graphml
    ox.save_graphml(G, filename='{studyregion}_{network_type}{suffix}.graphml'.format(
        studyregion = place, network_type=network_type, suffix = suffix), folder=folder)
    
    #set up project projection
    G_proj = ox.project_graph(G)
    edges_proj_gdfs = ox.graph_to_gdfs(G_proj, nodes=False, edges=True, fill_edge_geometry=True)
    
    #save projected network geodataframe as shapefile (project to UTM so we can use metres as a unit when buffering)
    ox.save_gdf_shapefile(edges_proj_gdfs, filename='{studyregion}_{network_type}{suffix}'.format(
        studyregion = place, network_type=network_type, suffix = suffix), folder=folder)
    
    #show network figure
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = edges_proj_gdfs.plot(ax=ax)
    ax.set_title(address)
    ax.set_axis_off()
    fig.suptitle('{} OSM {} street network'.format(place, network_type), fontsize=14, fontweight='bold')
    plt.show()



## Load pedestrain street network from local folder

In [6]:
def load_OSM_G(G_filename, OSM_folder):
    """
    Load a GraphML file from disk and convert the node/edge attributes to correct data types.

    Parameters
    ----------
    G_filename : string
        the name of the graphml file (including file extension)
    OSM_folder : string
        the folder containing the OSM file, if None, use default data folder

    Returns
    -------
    networkx multidigraph
    """
    G = ox.load_graphml(filename=G_filename, folder=OSM_folder)
    return G



In [9]:
def load_OSM_edges(shape_filepath):
    """
    Load a (projected) edge shapefile from disk 

    Parameters
    ----------
    shape_filepath : string
        the location of the shapefile
    
    Returns
    -------
    GeoDataFrame
    """
        
    edges = gpd.GeoDataFrame.from_file(shape_filepath)
    return edges

### Load OSM network stats
This function retains all the basic stats for all streets, pedestrain network, and cycle network within 10km buffered study regions. The OSM street network data are loaded from local data folder. 

In [8]:
def load_OSM_stats(G_filename, folder=OSM_folder):
    """
    retains all the basic stats for all streets, pedestrain network, and 
    cycle network within 10km buffered study regions from a local folder for OSM resource
    
    Parameters
    ----------
    polygon : shapely Polygon or MultiPolygon
        the shape to get network data within. coordinates should be in units of
        latitude-longitude degrees.
    network_type : string
        what type of street network to get. Default type is pedestrain network
    place: string
        place name
    suffix: string
        output data date
    folder : string
        where to save the shapefile, specify local folder path for OSM resource
    -------

    """
    df = pd.DataFrame()
    #load street network data from local directory
    for networktype in ['all', 'walk', 'bike']:
        G = ox.load_graphml(G_filename, folder=folder)
        
        gdf_nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
        graph_area_m = gdf_nodes_proj.unary_union.convex_hull.area
        
        stats = ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist='euclidean', tolerance=15)
        df1 = pd.DataFrame.from_dict(stats, orient='index', columns=[networktype + '_' + place])
        df = pd.concat([df, df1], axis=1)
    return df
    